#ADE-Net Attack Generation

Code and Paper by: Nicholas Soucy and Dr. Salimeh Yasaei Sekeh

##Initial Setup

### Imports

#### Importing

In [ ]:
#imports
import numpy as np
import keras
import keras.utils
from keras.utils.np_utils import to_categorical
from keras import layers
from keras.callbacks import TensorBoard
from sklearn.model_selection import ShuffleSplit
import h5py
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.special import softmax
from scipy.stats import entropy
import scipy.io as io
from keras.callbacks import ModelCheckpoint, Callback
import numpy as np
import time
import ipdb
import random
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from collections import Counter
from sklearn.metrics import cohen_kappa_score as kappa
import keras.backend as K
from keras.backend import clear_session
from sklearn.model_selection import cross_val_score
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint, Callback
from keras.layers import Conv2DTranspose, Add, Input, Concatenate, Layer, SeparableConv2D
from keras.layers import Dense, Activation, BatchNormalization, Dropout, LeakyReLU, Conv2D, Reshape
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from keras.layers import Conv3D, Conv3DTranspose, PReLU, BatchNormalization, MaxPool3D
from sklearn.model_selection import GridSearchCV
from keras.layers import Conv2DTranspose, Add, Input, Concatenate, Layer, SeparableConv2D
from keras.layers import Dense, Activation, BatchNormalization, Dropout, LeakyReLU, Conv2D, Reshape
from sklearn import preprocessing
from sklearn import utils
from sklearn import mixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import sklearn.preprocessing as sp
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as GMM
import keras
import os
import os.path as osp
import torch
import torch.utils.data as data
import yaml
import torch
import cv2
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
from torch import optim
from sklearn.metrics import confusion_matrix
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from operator import truediv
from plotly.offline import init_notebook_mode
import matplotlib.pyplot as plt
import scipy.io as sio
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras import models
import math
import scipy.io as io
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.python.ops.numpy_ops import np_config
from tqdm import tqdm
from tqdm import trange
import torch.utils.data as data_utils
import torchattacks
from scipy.sparse.linalg import LinearOperator as ScipyLinearOperator
from scipy.sparse.linalg import eigsh
from warnings import warn
import shutil
import sys

from art.estimators.classification import PyTorchClassifier
import art.attacks.evasion

if torch.cuda.is_available():
  print(torch.cuda.get_device_name(0))
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
np_config.enable_numpy_behavior()
drive.mount('/content/drive', force_remount=True)

In [ ]:
#GLOBAL VARIABLES TO CHANGE

#Dataset Stuff
dataset = ["Indian_Pines","Salinas","Pavia_University","Kennedy_Space_Center","Botswana","Houston"]
ds = 3

#Batch Number
batch = 1024

#Which epsilon we wish to test
epsilons = [0.1]
epi = 0

#Attack Stuff
attack_types = ['FGSM','PGD','BIM','CW']
num_attacks = 4

#The amount of training for each attack for discriminator
t_prime = 10

if ds == 0:
  PATH = "Data/Indian_Pines/"
elif ds == 1:
  PATH = "Data/Salinas/"
elif ds == 2:
  PATH = "Data/Pavia_University/"
elif ds == 3:
  PATH = "Data/Kennedy_Space_Center/"
elif ds == 4:
  PATH = "Data/Botswana/"
elif ds == 5:
  PATH = "Data/Houston/"

print("PATH: ",PATH)

### Dataset Functions

In [4]:
#importing dataset functions

def import_IP():
  #Number of Classes = 16
  #import indian Pines dataset
  dataset = io.loadmat('Data/Indian_Pines/Indian_pines_corrected.mat')
  data = dataset['indian_pines_corrected']

  groundtruth = io.loadmat('Data/Indian_Pines/Indian_pines_gt.mat')
  gt = groundtruth['indian_pines_gt']

  dim_x, dim_y, dim_z = data.shape
  dim = dim_x * dim_y

  x = np.zeros(shape=(dim, dim_z))

  cont = 0
  for i in range(dim_x):
      for j in range(dim_y):
          x[cont, :] = data[i, j, :]
          cont += 1

  #normalize x data
  scaler = sp.MinMaxScaler()
  x = scaler.fit_transform(x)

  X = x.reshape(dim_x, dim_y, dim_z)

  return X, gt

def import_Bot():
  #Number of Classes = 14
  #import Botswana dataset
  dataset = io.loadmat('Data/Botswana/Botswana.mat')
  data = dataset['Botswana']

  groundtruth = io.loadmat('Data/Botswana/Botswana_gt.mat')
  gt = groundtruth['Botswana_gt']

  dim_x, dim_y, dim_z = data.shape
  dim = dim_x * dim_y

  x = np.zeros(shape=(dim, dim_z))

  cont = 0
  for i in range(dim_x):
      for j in range(dim_y):
          x[cont, :] = data[i, j, :]
          cont += 1

  #normalize x data
  scaler = sp.MinMaxScaler()
  x = scaler.fit_transform(x)

  X = x.reshape(dim_x, dim_y, dim_z)

  return X, gt

def import_KSC():
  #Number of classes = 13
  #import Kennedy Space Center dataset
  dataset = io.loadmat('Data/Kennedy_Space_Center/KSC.mat')
  data = dataset['KSC']

  groundtruth = io.loadmat('Data/Kennedy_Space_Center/KSC_gt.mat')
  gt = groundtruth['KSC_gt']

  dim_x, dim_y, dim_z = data.shape
  dim = dim_x * dim_y

  x = np.zeros(shape=(dim, dim_z))

  cont = 0
  for i in range(dim_x):
      for j in range(dim_y):
          x[cont, :] = data[i, j, :]
          cont += 1

  #normalize x data
  scaler = sp.MinMaxScaler()
  x = scaler.fit_transform(x)

  X = x.reshape(dim_x, dim_y, dim_z)

  return X, gt

def import_Sal():
  #Number of classes = 16
  #import Salinas dataset
  dataset = io.loadmat('Data/Salinas/Salinas_corrected.mat')
  data = dataset['salinas_corrected']

  groundtruth = io.loadmat('Data/Salinas/Salinas_gt.mat')
  gt = groundtruth['salinas_gt']

  dim_x, dim_y, dim_z = data.shape
  dim = dim_x * dim_y

  x = np.zeros(shape=(dim, dim_z))

  cont = 0
  for i in range(dim_x):
      for j in range(dim_y):
          x[cont, :] = data[i, j, :]
          cont += 1

  #normalize x data
  scaler = sp.MinMaxScaler()
  x = scaler.fit_transform(x)

  X = x.reshape(dim_x, dim_y, dim_z)

  return X, gt

def import_PavU():
  #Number of classes = 9
  #import Pavia University dataset
  dataset = io.loadmat('Data/Pavia_University/PaviaU.mat')
  data = dataset['paviaU']

  groundtruth = io.loadmat('Data/Pavia_University/PaviaU_gt.mat')
  gt = groundtruth['paviaU_gt']

  dim_x, dim_y, dim_z = data.shape
  dim = dim_x * dim_y

  x = np.zeros(shape=(dim, dim_z))

  cont = 0
  for i in range(dim_x):
      for j in range(dim_y):
          x[cont, :] = data[i, j, :]
          cont += 1

  #normalize x data
  scaler = sp.MinMaxScaler()
  x = scaler.fit_transform(x)

  X = x.reshape(dim_x, dim_y, dim_z)

  return X, gt

def import_Houston():
  #Number of Classes = 15
  #import Houston dataset
  dataset = io.loadmat('Data/Houston/Houston.mat')
  data = dataset['Houston']

  groundtruth = io.loadmat('Data/Houston/Houston_gt.mat')
  gt = groundtruth['gt']

  dim_x, dim_y, dim_z = data.shape
  dim = dim_x * dim_y

  x = np.zeros(shape=(dim, dim_z))

  cont = 0
  for i in range(dim_x):
      for j in range(dim_y):
          x[cont, :] = data[i, j, :]
          cont += 1

  #normalize x data
  scaler = sp.MinMaxScaler()
  x = scaler.fit_transform(x)

  X = x.reshape(dim_x, dim_y, dim_z)

  return X, gt


#Data reshaping Functions

def remove_background(data, gt):
  tmp_a = []
  for i in range(data.shape[0]):
    if gt[i] == 0:
      tmp_a.append(i)

  x = np.delete(data,tmp_a,0)
  y = np.delete(gt,tmp_a,0)
  return x, y

def flatten(data, gt):
  #reshape data
  dim_x, dim_y, dim_z = data.shape
  dim = dim_x * dim_y

  x = np.zeros(shape=(dim, dim_z))
  y = gt


  cont = 0
  for i in range(dim_x):
      for j in range(dim_y):
          x[cont, :] = data[i, j, :]
          cont += 1

  y = np.reshape(y,(dim))


  #normalize x data
  scaler = sp.MinMaxScaler()
  x = scaler.fit_transform(x)

  return x, y


In [5]:
#functions and classes for our custom U-Net

def process_data(data,gt,class_num = 16):

  def load_patches(data, gt, patch_size = 1):

      patches = []
      patches_gt = []
      for i in range(data.shape[0] - patch_size):
        for j in range(data.shape[1] - patch_size):
            patch = (data[i:i+patch_size, j:j+patch_size, :]).copy()
            patch_gt = (gt[i:i+patch_size, j:j+patch_size]).copy()
            if np.any(patch_gt == 0):
                continue
            else:
                patches.append(patch)
                patches_gt.append(patch_gt)
      patches_1 = np.array(patches)
      patches_gt = np.array(patches_gt) - 1
      patches_gt_1 = to_categorical(patches_gt, num_classes = class_num)
      patches_gt_2 = patches_gt_1.reshape(patches_gt_1.shape[0],1,1,patches_gt_1.shape[2])
      return patches_1, patches_gt_2

  def load_synthetic_patches(data, gt, patch_size = 1, small_patch_size = 1, 
                            oversample = 12, label_choice = 8):

      patches_small = [[] for _ in range(class_num)] #16 classes
      patches_gt_small = [[] for _ in range(class_num)]

      for i in range(data.shape[0] - small_patch_size):
        for j in range(data.shape[1] - small_patch_size):
            patch = (data[i:i+small_patch_size, j:j+small_patch_size, :]).copy()
            patch_gt = (gt[i:i+small_patch_size, j:j+small_patch_size]).copy()
            if np.any(patch_gt == 0):
                continue
            else:
                index = patch_gt[0,0] - 1
                patches_small[index].append(patch)
                patches_gt_small[index].append(patch_gt)

      patches_small = [np.array(patches_small[i]) for i in range(class_num)]
      patches_gt_small = [(np.array(patches_gt_small[i]) - 1) for i in range(class_num)]

      ## Mixed patches

      patches = []
      patches_gt = []

      for sample in range(int(oversample)):
        new_patch = np.zeros((patch_size, patch_size, data.shape[2]))
        new_patch_gt = np.zeros((patch_size, patch_size))
                
        for i in range(0, patch_size, small_patch_size):
            for j in range(0, patch_size, small_patch_size):
            
                index_choice = np.random.randint(int(len(patches_small[label_choice]) * 0.75))
                new_patch[i:i+small_patch_size, j:j+small_patch_size, :] = patches_small[label_choice][index_choice]
                new_patch_gt[i:i+small_patch_size, j:j+small_patch_size] = patches_gt_small[label_choice][index_choice]            

        patches.append(new_patch)
        patches_gt.append(new_patch_gt)

      patches = np.array(patches)
      patches_gt = np.array(patches_gt)
      patches_gt = to_categorical(patches_gt, num_classes=class_num)

      return patches, patches_gt

  for i in range(data.shape[-1]):
    data[:,:,i] = (data[:,:,i] - np.mean(data[:,:,i])) / np.std(data[:,:,i])
  train_patches, train_patches_gt = load_patches(data, gt)
  train_patches_synthetic, train_patches_gt_synthetic = load_synthetic_patches(data, gt)
  train_patches_gt_synthetic = train_patches_gt_synthetic.reshape(train_patches_gt_synthetic.shape[0],1,1,train_patches_gt_synthetic.shape[2])
  train_patches = np.concatenate((train_patches, train_patches_synthetic), axis=0)
  train_patches_gt = np.concatenate((train_patches_gt, train_patches_gt_synthetic), axis=0)

  return (train_patches, train_patches_gt)

class PixelSoftmax(Layer):
    """
    Pixelwise Softmax for Semantic Segmentation. Also known as
    4D Softmax in some sources. Applies Softmax along the last
    axis (-1 axis). 
    """
    def __init__(self, axis=-1, **kwargs):
        self.axis=axis
        super(PixelSoftmax, self).__init__(**kwargs)

    def get_config(self):
      config = super().get_config().copy()
      return config

    def build(self,input_shape):
        pass

    def call(self, x,mask=None):
        e = K.exp(x - K.max(x, axis=self.axis, keepdims=True))
        s = K.sum(e, axis=self.axis, keepdims=True)
        return e / s

    def get_output_shape_for(self, input_shape):
        return input_shape

class statsLogger(Callback):
    """
    Saving loss and accuracy details to an array
    """
    def __init__(self):
        self.logs = []
    def on_epoch_end(self, epoch, logs):
        logs['epoch'] = epoch
        self.logs.append(logs)
    def get_config(self):
        config = super().get_config().copy()
        return config


## Feature Reduction Functions

### PCA

In [6]:
#PCA

def PCA_DR(X,n):
  #PCA
  pca = PCA(n_components = n)

  principalComponents = pca.fit_transform(X)

  ev=pca.explained_variance_ratio_

  return principalComponents

## Classification Functions

### Unet

#### Pytorch Unet Class

Actual Custom Neural Network Class

In [7]:
class CustomNeuralNetwork(nn.Module):
    def __init__(self,in_shape,class_num,want_logits=True):
        super().__init__()
        # Define Variables
        self.input_shape = in_shape
        self.class_num = class_num
        self.want_logits = want_logits
        # Define Layers:
        # NOTE: Change kernel size to be the size of the patch
        self.Conv1 = nn.Conv2d(self.input_shape, 64,kernel_size=(1,1), stride=(1,1), padding='same', bias=False)
        self.bn1 = nn.InstanceNorm1d(64)
        self.Conv2 = nn.Conv2d(64, 128,kernel_size=(1,1), stride=(2,2), bias=False)
        self.bn2 = nn.InstanceNorm1d(128)
        self.Conv3 = nn.Conv2d(128, 256,kernel_size=(1,1), stride=(2,2), bias=False)
        self.bn3 = nn.InstanceNorm1d(256)
        self.deco1 = nn.ConvTranspose2d(256,256,kernel_size=(1,1), stride=(1,1), bias=False)
        self.deco2 = nn.ConvTranspose2d(384,128,kernel_size=(1,1), stride=(1,1), bias=False)
        self.deco3 = nn.ConvTranspose2d(192,class_num,kernel_size=(1,1), stride=(1,1))

        # Define Activation functions:
        self.drop = nn.Dropout(0.2)
        self.Lrelu = nn.LeakyReLU()
        self.softmax = nn.Softmax(dim=-1)
    def forward(self, x):
        """
        Layers: 4
        Activation Functions:
        RELU for first two layers
        Sigmoid for third layer
        Log Softmax for last layer
        """
        # import pdb;pdb.set_trace()
        x = self.Conv1(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn1(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        op1 = x

        x = self.Conv2(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn2(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        op2 = x

        x = self.Conv3(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn3(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        op3 = x

        x = self.deco1(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn3(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        x = torch.cat((x, op2), dim=1)

        x = self.deco2(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn2(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        x = torch.cat((x, op1), dim=1)

        x = self.deco3(x)
        x = torch.reshape(x, (x.shape[0], self.class_num))

        if(self.want_logits):
          return x
        else:
          x = self.softmax(x)
          return x

#### Adversarial Generation Network

Original network for adversairal generation via categorical cross entropy class loss

In [8]:
def accuracy(output, target, topk=(1,)):
  """Computes the precision@k for the specified values of k"""
  maxk = max(topk)
  batch_size = target.size(0)

  _, pred = output.topk(maxk, 1, True, True)
  pred = pred.t()
  correct = pred.eq(target.view(1, -1).expand_as(pred))

  res = []
  for k in topk:
      correct_k = correct[:k].view(-1).float().sum(0)
      res.append(correct_k.mul_(100.0 / batch_size))
  return res


def UNet_Conv_torch(train_loader, test_loader=None, num_epochs = 25, class_num = 16, want_logits = True, return_loss = False, return_train_acc = False, trained_model = None):

  for i, (images, labels) in enumerate(train_loader):
    in_shape = images.shape[1]
  
  if trained_model == None:
    NN = CustomNeuralNetwork(in_shape,class_num)  # Intialize you NN
  else:
    NN = trained_model
  NN = NN.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(NN.parameters(), lr = 0.001)
  min_train_loss = 10000000

  for epoch in trange(num_epochs,desc="Training Pytorch U-Net"):
      train_running_loss = 0.0
      train_acc = 0.0

      model = NN.train()

      ## training step
      for idx, (images, labels) in enumerate(train_loader):

          images = images.clone().detach().requires_grad_(True)
          labels = labels.clone().detach().requires_grad_(True)

          images = images.to(device)
          labels = labels.to(device)

          ## forward + backprop + loss
          output = model(images)
          loss = criterion(output, labels)
          optimizer.zero_grad()
          loss.backward()

          ## update model params
          optimizer.step()

          train_running_loss += loss.detach().item()

          # _, predicted = torch.max(output.data, 1)
          # total = labels.size(0)
          # correct = (predicted == labels).sum().item()
          # train_acc = (100 * correct / total)
      
      model.eval()
      print('Epoch: %d | Loss: %.4f' \
            %(epoch, train_running_loss / idx))  
      if (train_running_loss / idx) < min_train_loss:
        min_train_loss = (train_running_loss / idx)



  # Accuracy on Test Data
  correct = 0
  total = 0
  if test_loader != None:
    loader = test_loader
  else:
    loader = train_loader
    flag = True
  with torch.no_grad():
      for idx, (images, labels) in enumerate(loader):

          images = images.clone().detach().requires_grad_(True)
          if flag:
            labels = torch.tensor(np.array(tf.math.argmax(labels.numpy(), axis=1)))
          else:
            labels = labels.clone().detach().requires_grad_(True)

          images = images.to(device)
          labels = labels.to(device)

          output = model(images)
          # import pdb;pdb.set_trace()
          # acc = accuracy(output, labels)
          _, predicted = torch.max(output.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  # return(acc,model)
  if return_loss and return_train_acc:
    return((100 * correct / total),model,loss,min_train_loss)
  else:
    return((100 * correct / total),model,optimizer)

## Training for Adversarial Generation


Train U-Net for Adversarial Generation

In [ ]:
if ds == 0:
  X, Y = import_IP()
  cls = 16
  en_type = 'k'
  clusters = 2
  print(dataset[ds])
  print(X.shape, Y.shape)
elif ds == 1:
  X, Y = import_Sal()
  cls = 16
  en_type = 'g'
  clusters = 3
  print(dataset[ds])
  print(X.shape, Y.shape)
elif ds == 2: 
  X, Y = import_PavU()
  cls = 9
  en_type = 'g'
  clusters = 2
  print(dataset[ds])
  print(X.shape, Y.shape)
elif ds == 3:
  X, Y = import_KSC()
  cls = 13
  en_type = 'k'
  clusters = 2
  print(dataset[ds])
  print(X.shape, Y.shape)
elif ds == 4:
  X, Y = import_Bot()
  cls = 14
  en_type = 'k'
  clusters = 3
  print(dataset[ds])
  print(X.shape, Y.shape)
elif ds == 5:
  X, Y = import_Houston()
  cls = 15
  print(dataset[ds])
  print(X.shape, Y.shape)

#Apply PCA
X1 = X.shape[0]
X2 = X.shape[1]
X_f = X.reshape(X.shape[0]*X.shape[1],X.shape[-1])
X = PCA_DR(X_f,30)
X = X.reshape(X1,X2,X.shape[-1])

#Process data for U-Net
data, gt = process_data(X, Y, class_num = cls)
print(data.shape, gt.shape)

#convert data to correct shape
gt = np.reshape(gt, (-1, cls))
data = data.reshape(data.shape[0],data.shape[-1],data.shape[1],data.shape[2])

In [ ]:
#convert to Tensors
gt = torch.Tensor(np.array(gt))
data = torch.Tensor(np.array(data))

#create datasets
dataset = data_utils.TensorDataset(data, gt)
train_loader = data_utils.DataLoader(dataset, batch_size=batch, shuffle=False,pin_memory=True)
# dataset2 = data_utils.TensorDataset(x_test, y_test)
# test_loader = data_utils.DataLoader(dataset2, batch_size=batch, shuffle=False,pin_memory=True)


acc, model, optimizer = UNet_Conv_torch(train_loader, num_epochs = 150, class_num = cls)



In [ ]:
print(acc)

## Adversarial Generation

Function to apply attacks to a range of data

In [12]:
def Apply_Attacks(data,gt,model,optimizer,a_type='FGSM',cls = 16):

  data = np.array(data)
  print("gt.shape: ",gt.shape)
  gt = np.argmax(np.array(gt),axis=1)
  print("gt.shape now: ",gt.shape)

  classifier = PyTorchClassifier(model=model, clip_values=(0.0, 1.0), loss=nn.CrossEntropyLoss(), optimizer=optimizer, input_shape=(30, 1, 1), nb_classes=cls)
            
  if a_type == 'FGSM':
    atk = art.attacks.evasion.FastGradientMethod(estimator=classifier, eps=.1)
  elif a_type == 'BIM':
    atk = art.attacks.evasion.BasicIterativeMethod(estimator=classifier, eps=.1)
  elif a_type == 'PGD':
    atk = art.attacks.evasion.ProjectedGradientDescentPyTorch(estimator=classifier, eps=.1)
  elif a_type == 'CW':
    atk = art.attacks.evasion.CarliniL2Method(classifier=classifier)

  perturbations = atk.generate(data)
  perturbations = np.asarray(perturbations)
    
  return perturbations, gt
  

Attack the full x_test an 'num_attacks' amount of times and put results into array

In [ ]:
print(data.shape)
print(gt.shape)

In [ ]:
attacks_used = [attack_types[0],attack_types[1],attack_types[2],attack_types[3]]
names = []
data_att = [None] * num_attacks
gt_att = [None] * num_attacks

for i in range(num_attacks):
  print("On Number ",(i+1))

  data_att[i], gt_att[i] = Apply_Attacks(data.cpu().detach().numpy(),gt.cpu().detach().numpy(),model,optimizer,a_type=attacks_used[i],cls = cls)

  names.append(attacks_used[i])
  print(names[i])

In [ ]:
for i in range(num_attacks):
  print(data_att[i].shape)

attack_types = ['VANILA','FGSM','PGD','BIM','CW']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_att[0], gt_att[0], test_size=0.25, random_state=42)
print(names[0])
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
np.save("{}VANILLA/x_train".format(PATH), x_train)
np.save("{}VANILLA/x_test".format(PATH), x_test)
np.save("{}VANILLA/y_train".format(PATH), y_train)
np.save("{}VANILLA/y_test".format(PATH), y_test)

In [ ]:
x_adv_train, x_adv_test, y_train, y_test = train_test_split(data_att[0], gt_att[0], test_size=0.25, random_state=42)
print(names[0])
print(x_adv_train.shape)
print(x_adv_test.shape)
print(y_train.shape)
print(y_test.shape)
np.save("{}FGSM/x_adv_train".format(PATH), x_adv_train)
np.save("{}FGSM/x_adv_test".format(PATH), x_adv_test)
np.save("{}FGSM/y_train".format(PATH), y_train)
np.save("{}FGSM/y_test".format(PATH), y_test)

In [ ]:
x_adv_train, x_adv_test, y_train, y_test = train_test_split(data_att[1], gt_att[1], test_size=0.25, random_state=42)
print(names[1])
print(x_adv_train.shape)
print(x_adv_test.shape)
print(y_train.shape)
print(y_test.shape)
np.save("{}PGD/x_adv_train".format(PATH), x_adv_train)
np.save("{}PGD/x_adv_test".format(PATH), x_adv_test)
np.save("{}PGD/y_train".format(PATH), y_train)
np.save("{}PGD/y_test".format(PATH), y_test)

In [ ]:
x_adv_train, x_adv_test, y_train, y_test = train_test_split(data_att[2], gt_att[2], test_size=0.25, random_state=42)
print(names[2])
print(x_adv_train.shape)
print(x_adv_test.shape)
print(y_train.shape)
print(y_test.shape)
np.save("{}I-FGSM/x_adv_train".format(PATH), x_adv_train)
np.save("{}I-FGSM/x_adv_test".format(PATH), x_adv_test)
np.save("{}I-FGSM/y_train".format(PATH), y_train)
np.save("{}I-FGSM/y_test".format(PATH), y_test)

In [ ]:
x_adv_train, x_adv_test, y_train, y_test = train_test_split(data_att[3], gt_att[3], test_size=0.25, random_state=42)
print(names[3])
print(x_adv_train.shape)
print(x_adv_test.shape)
print(y_train.shape)
print(y_test.shape)
np.save("{}CW/x_adv_train".format(PATH), x_adv_train)
np.save("{}CW/x_adv_test".format(PATH), x_adv_test)
np.save("{}CW/y_train".format(PATH), y_train)
np.save("{}CW/y_test".format(PATH), y_test)